<a href="https://colab.research.google.com/github/samantamrityunjay/ICDcoding/blob/master/NonAI_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import tqdm 
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
from ast import literal_eval
# import nltk
# nltk.download('punkt')
# from nltk.tokenize import word_tokenize
# import string
pd.set_option("display.max_colwidth",None)

In [ ]:
from gensim.models import KeyedVectors
%time model = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/BioVecModels/biowordvec", binary=True,limit=1000000)

CPU times: user 14.5 s, sys: 2.37 s, total: 16.9 s
Wall time: 26.8 s


In [ ]:
frequent_icd9category=['401','427','276','414','272','250','428','518','285','584']

In [ ]:
diagnosis_icd9category_token=pd.read_csv("/content/drive/My Drive/AlgoIntern/Data/Data2.0/diagnosis_icd9category1.csv")

In [ ]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","['496', '553', '518', '453', '519', '276']","['hypercholesterolemia', 'cerebral', 'hypothyroidism', 'cough', 'accident', 'hypertension', 'artery', 'chest', 'vascular', 'coronary', 'fracture', 'injury', 'angina', 'pain', 'head']"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","['487', '416', '272', '530', '414', '553', '531', '518', '401', '519', '348', 'V02', '359', '584', '458', '578', '491', 'E93', '276', '244']","['pulmonary', 'hypertension', 'of', 'obstructive', 'discomfort', 'gastrointestinal', 'constipation', 'oxygen', 'breath', 'inflammation', 'artery', 'chest', 'hydrochlorothiazide', 'steroid', 'anxiety', 'omeprazole', 'senna', 'trazodone', 'failure', 'shortness', 'disease', 'pantoprazole', 'aspirin', 'hypothyroidism', 'zolpidem', 'furosemide', 'coronary', 

In [ ]:
diagnosis_icd9category_token['medical_terms_list']=diagnosis_icd9category_token.medical_terms_list.apply(literal_eval)
diagnosis_icd9category_token['icd9_category']=diagnosis_icd9category_token.icd9_category.apply(literal_eval)

In [ ]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518, 401, 519, 348, V02, 359, 584, 458, 578, 491, E93, 276, 244]","[pulmonary, hypertension, of, obstructive, discomfort, gastrointestinal, constipation, oxygen, breath, inflammation, artery, chest, hydrochlorothiazide, steroid, anxiety, omeprazole, senna, trazodone, failure, shortness, disease, pantoprazole, aspirin, hypothyroidism, zolpidem, furosemide, coronary, prednisone, lethargy, steroids, ulcer, chronic, respiratory, morphine, pain]"
2,135453.0,discharge diagnosis: 1. cervical spondylosis wi

In [ ]:
category_med_terms={}
common_diagnosis_terms=["monday","telephone","of","phone"]
for i in frequent_icd9category:
    diagnosis_x=diagnosis_icd9category_token.loc[diagnosis_icd9category_token.icd9_category.apply(lambda x: i in x)].copy()
    y=diagnosis_x.medical_terms_list.to_list()    
    z=[list(set(a)) for a in y]
    z1=[a for b in z for a in b]
    dict_med=dict(Counter(z1))
    dict_med=dict(sorted(dict_med.items(),key= lambda x: x[1],reverse=True))
    dict_med_list=list(dict_med)
    dict_med_list1=[a for a in dict_med_list if a not in common_diagnosis_terms]
    category_med_terms[i]=dict_med_list1[:50]

diagnosis_icdcategory_medterms={'icd9category':[i for i in category_med_terms],
                                'frequent_medterms':[category_med_terms[i] for i in category_med_terms]}
diagnosis_icdcategory_medterms_df=pd.DataFrame.from_dict(diagnosis_icdcategory_medterms)

diagnosis_icdcategory_medterms_df

In [ ]:
y=diagnosis_icdcategory_medterms_df.frequent_medterms.to_list()
y1=[a for b in y for a in b]
medical=dict(Counter(y1))
print([k for (k,v) in medical.items() if v>=16])

[]


In [ ]:
bagofwords=list(set([a for b in diagnosis_icdcategory_medterms_df.frequent_medterms.to_list() for a in b]))

In [ ]:
len(bagofwords)

76

In [ ]:
set(bagofwords).issubset(model.vocab)

True

In [ ]:
simi_matrix=np.zeros((len(bagofwords),len(bagofwords)))

In [ ]:
simi_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
simi_matrix.shape

(76, 76)

In [ ]:
for i in tqdm(range(len(bagofwords))):
  for j in range(len(bagofwords)):
    simi_matrix[i][j]=model.similarity(bagofwords[i],bagofwords[j])

100%|██████████| 76/76 [00:00<00:00, 783.53it/s]


In [ ]:
simi_matrix

array([[0.99999994, 0.25552419, 0.32569456, ..., 0.34229475, 0.30997536,
        0.3215678 ],
       [0.25552419, 0.99999994, 0.26996946, ..., 0.36623082, 0.38000947,
        0.30598155],
       [0.32569456, 0.26996946, 1.        , ..., 0.27386397, 0.4312855 ,
        0.41733652],
       ...,
       [0.34229475, 0.36623082, 0.27386397, ..., 0.99999994, 0.2215088 ,
        0.28342548],
       [0.30997536, 0.38000947, 0.4312855 , ..., 0.2215088 , 1.        ,
        0.38060111],
       [0.3215678 , 0.30598155, 0.41733652, ..., 0.28342548, 0.38060111,
        1.        ]])

In [ ]:
features={}
for i in range(len(bagofwords)-1):
  similar=[]
  for j in range(i+1,len(bagofwords)):
    if simi_matrix[i][j]>=0.7:
      similar.append(bagofwords[j])
  features[bagofwords[i]]=similar
features[bagofwords[-1]]=[]    


In [ ]:
print(features)

{'fibrillation': ['atrial'], 'blood': [], 'alcohol': [], 'fracture': [], 'constipation': ['vomiting', 'diarrhea', 'nausea'], 'seizures': [], 'lung': ['pulmonary'], 'tylenol': ['motrin'], 'myocardial': ['coronary', 'infarction'], 'numbness': ['pain', 'weakness', 'dizziness'], 'hyperlipidemia': ['hypertension', 'mellitus'], 'systolic': [], 'swelling': [], 'stroke': ['infarction'], 'redness': ['erythema'], 'infection': [], 'lisinopril': ['metoprolol'], 'oxygen': [], 'coronary': ['artery'], 'prednisone': [], 'vomiting': ['chills', 'diarrhea', 'dizziness', 'nausea'], 'warfarin': ['aspirin'], 'aspirin': [], 'infarction': [], 'hypertension': [], 'failure': ['congestive'], 'gain': [], 'loss': [], 'kidney': ['renal'], 'acute': ['chronic'], 'respiratory': [], 'obstructive': [], 'pain': ['headache'], 'motrin': [], 'dehydration': [], 'sepsis': ['pneumonia'], 'shortness': ['chills', 'fevers'], 'pneumonia': [], 'abdominal': [], 'atrial': [], 'breath': [], 'chills': ['fever', 'dizziness', 'nausea', '

In [ ]:
len(features)

76

In [ ]:
features_list=[features[k]+[k] for k in list(features)]

In [ ]:
features_list

[['atrial', 'fibrillation'],
 ['blood'],
 ['alcohol'],
 ['fracture'],
 ['vomiting', 'diarrhea', 'nausea', 'constipation'],
 ['seizures'],
 ['pulmonary', 'lung'],
 ['motrin', 'tylenol'],
 ['coronary', 'infarction', 'myocardial'],
 ['pain', 'weakness', 'dizziness', 'numbness'],
 ['hypertension', 'mellitus', 'hyperlipidemia'],
 ['systolic'],
 ['swelling'],
 ['infarction', 'stroke'],
 ['erythema', 'redness'],
 ['infection'],
 ['metoprolol', 'lisinopril'],
 ['oxygen'],
 ['artery', 'coronary'],
 ['prednisone'],
 ['chills', 'diarrhea', 'dizziness', 'nausea', 'vomiting'],
 ['aspirin', 'warfarin'],
 ['aspirin'],
 ['infarction'],
 ['hypertension'],
 ['congestive', 'failure'],
 ['gain'],
 ['loss'],
 ['renal', 'kidney'],
 ['chronic', 'acute'],
 ['respiratory'],
 ['obstructive'],
 ['headache', 'pain'],
 ['motrin'],
 ['dehydration'],
 ['pneumonia', 'sepsis'],
 ['chills', 'fevers', 'shortness'],
 ['pneumonia'],
 ['abdominal'],
 ['atrial'],
 ['breath'],
 ['fever', 'dizziness', 'nausea', 'fevers', 'chi

In [ ]:
len(features_list)

76

In [ ]:
#Python code to merge all sublist having common elements. 

#Importing 
from collections import defaultdict 

#merge function to merge all sublist having common elements. 
def merge_common(lists): 
	neigh = defaultdict(set) 
	visited = set() 
	for each in lists: 
		for item in each: 
			neigh[item].update(each) 
	def comp(node, neigh = neigh, visited = visited, vis = visited.add): 
		nodes = set([node]) 
		next_node = nodes.pop 
		while nodes: 
			node = next_node() 
			vis(node) 
			nodes |= neigh[node] - visited 
			yield node 
	for node in neigh: 
		if node not in visited: 
			yield sorted(comp(node)) 

# #Input list initialization 
# Input = [['z','x','y'], ['y','g','e'], ['z'],['x','p'], 
# 		['a','b'], ['y','a'], ['d','g']] 

#Calling merge function 
condensed_feature = list(merge_common(features_list)) 

#Printing function 
# print("Initial list of list is :") 
# print(Input) 

# print("List of list after merging is:") 
# print(Output) 


In [ ]:
condensed_feature

[['atrial', 'fibrillation'],
 ['blood'],
 ['alcohol'],
 ['fracture'],
 ['chills',
  'constipation',
  'diarrhea',
  'dizziness',
  'fever',
  'fevers',
  'headache',
  'nausea',
  'numbness',
  'pain',
  'shortness',
  'vomiting',
  'weakness'],
 ['seizures'],
 ['lung', 'pulmonary'],
 ['motrin', 'tylenol'],
 ['artery', 'coronary', 'infarction', 'myocardial', 'stroke'],
 ['diabetes', 'hyperlipidemia', 'hypertension', 'mellitus'],
 ['systolic'],
 ['swelling'],
 ['erythema', 'redness'],
 ['infection'],
 ['lisinopril', 'metoprolol'],
 ['oxygen'],
 ['prednisone'],
 ['aspirin', 'warfarin'],
 ['congestive', 'failure'],
 ['gain'],
 ['loss'],
 ['kidney', 'renal'],
 ['acute', 'chronic'],
 ['respiratory'],
 ['obstructive'],
 ['dehydration'],
 ['pneumonia', 'sepsis'],
 ['abdominal'],
 ['breath'],
 ['mouth'],
 ['urinary'],
 ['confusion'],
 ['tract'],
 ['cough'],
 ['bleed', 'bleeding'],
 ['coumadin'],
 ['edema'],
 ['chest'],
 ['disease'],
 ['anemia'],
 ['gastrointestinal'],
 ['heart'],
 ['weight'],


In [ ]:
len(condensed_feature)

46

In [ ]:
model.similarity('abdominal','edema')

0.40815303

In [ ]:
model.similarity('gain','loss')

0.6596458

In [ ]:
condensed_feature_dict={}
count=0
for i in condensed_feature:
  condensed_feature_dict['f'+str(count)]=i
  count+=1

In [ ]:
print(condensed_feature_dict)

{'f0': ['atrial', 'fibrillation'], 'f1': ['blood'], 'f2': ['alcohol'], 'f3': ['fracture'], 'f4': ['chills', 'constipation', 'diarrhea', 'dizziness', 'fever', 'fevers', 'headache', 'nausea', 'numbness', 'pain', 'shortness', 'vomiting', 'weakness'], 'f5': ['seizures'], 'f6': ['lung', 'pulmonary'], 'f7': ['motrin', 'tylenol'], 'f8': ['artery', 'coronary', 'infarction', 'myocardial', 'stroke'], 'f9': ['diabetes', 'hyperlipidemia', 'hypertension', 'mellitus'], 'f10': ['systolic'], 'f11': ['swelling'], 'f12': ['erythema', 'redness'], 'f13': ['infection'], 'f14': ['lisinopril', 'metoprolol'], 'f15': ['oxygen'], 'f16': ['prednisone'], 'f17': ['aspirin', 'warfarin'], 'f18': ['congestive', 'failure'], 'f19': ['gain'], 'f20': ['loss'], 'f21': ['kidney', 'renal'], 'f22': ['acute', 'chronic'], 'f23': ['respiratory'], 'f24': ['obstructive'], 'f25': ['dehydration'], 'f26': ['pneumonia', 'sepsis'], 'f27': ['abdominal'], 'f28': ['breath'], 'f29': ['mouth'], 'f30': ['urinary'], 'f31': ['confusion'], 'f3

In [ ]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518, 401, 519, 348, V02, 359, 584, 458, 578, 491, E93, 276, 244]","[pulmonary, hypertension, of, obstructive, discomfort, gastrointestinal, constipation, oxygen, breath, inflammation, artery, chest, hydrochlorothiazide, steroid, anxiety, omeprazole, senna, trazodone, failure, shortness, disease, pantoprazole, aspirin, hypothyroidism, zolpidem, furosemide, coronary, prednisone, lethargy, steroids, ulcer, chronic, respiratory, morphine, pain]"
2,135453.0,discharge diagnosis: 1. cervical spondylosis wi

In [ ]:
condensed_feature

[['atrial', 'fibrillation'],
 ['blood'],
 ['alcohol'],
 ['fracture'],
 ['chills',
  'constipation',
  'diarrhea',
  'dizziness',
  'fever',
  'fevers',
  'headache',
  'nausea',
  'numbness',
  'pain',
  'shortness',
  'vomiting',
  'weakness'],
 ['seizures'],
 ['lung', 'pulmonary'],
 ['motrin', 'tylenol'],
 ['artery', 'coronary', 'infarction', 'myocardial', 'stroke'],
 ['diabetes', 'hyperlipidemia', 'hypertension', 'mellitus'],
 ['systolic'],
 ['swelling'],
 ['erythema', 'redness'],
 ['infection'],
 ['lisinopril', 'metoprolol'],
 ['oxygen'],
 ['prednisone'],
 ['aspirin', 'warfarin'],
 ['congestive', 'failure'],
 ['gain'],
 ['loss'],
 ['kidney', 'renal'],
 ['acute', 'chronic'],
 ['respiratory'],
 ['obstructive'],
 ['dehydration'],
 ['pneumonia', 'sepsis'],
 ['abdominal'],
 ['breath'],
 ['mouth'],
 ['urinary'],
 ['confusion'],
 ['tract'],
 ['cough'],
 ['bleed', 'bleeding'],
 ['coumadin'],
 ['edema'],
 ['chest'],
 ['disease'],
 ['anemia'],
 ['gastrointestinal'],
 ['heart'],
 ['weight'],


In [ ]:
all_vocab=[a for b in condensed_feature for a in b]

In [ ]:
all_vocab

['atrial',
 'fibrillation',
 'blood',
 'alcohol',
 'fracture',
 'chills',
 'constipation',
 'diarrhea',
 'dizziness',
 'fever',
 'fevers',
 'headache',
 'nausea',
 'numbness',
 'pain',
 'shortness',
 'vomiting',
 'weakness',
 'seizures',
 'lung',
 'pulmonary',
 'motrin',
 'tylenol',
 'artery',
 'coronary',
 'infarction',
 'myocardial',
 'stroke',
 'diabetes',
 'hyperlipidemia',
 'hypertension',
 'mellitus',
 'systolic',
 'swelling',
 'erythema',
 'redness',
 'infection',
 'lisinopril',
 'metoprolol',
 'oxygen',
 'prednisone',
 'aspirin',
 'warfarin',
 'congestive',
 'failure',
 'gain',
 'loss',
 'kidney',
 'renal',
 'acute',
 'chronic',
 'respiratory',
 'obstructive',
 'dehydration',
 'pneumonia',
 'sepsis',
 'abdominal',
 'breath',
 'mouth',
 'urinary',
 'confusion',
 'tract',
 'cough',
 'bleed',
 'bleeding',
 'coumadin',
 'edema',
 'chest',
 'disease',
 'anemia',
 'gastrointestinal',
 'heart',
 'weight',
 'cancer',
 'cholesterol',
 'sodium']

In [ ]:
diagnosis_icd9category_token['med_terms_condensed']=diagnosis_icd9category_token.medical_terms_list.progress_apply(lambda x:list(set(x).intersection(set(all_vocab))))

In [ ]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_condensed
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[fracture, coronary, chest, hypertension, artery, cough, pain]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518, 401, 519, 348, V02, 359, 584, 458, 578, 491, E93, 276, 244]","[pulmonary, hypertension, of, obstructive, discomfort, gastrointestinal, constipation, oxygen, breath, inflammation, artery, chest, hydrochlorothiazide, steroid, anxiety, omeprazole, senna, trazodone, failure, shortness, disease, pantoprazole, aspirin, hypothyroidism, zolpidem, furosemide, coronary, prednisone, lethargy, steroids, ulcer, chronic, res

In [ ]:
def return_feature(x):
  feature=[]
  for i in x:
    for j in list(condensed_feature_dict):
      if i in condensed_feature_dict[j]:
        feature.append(j)
  return list(set(feature))


In [ ]:
diagnosis_icd9category_token['features']=diagnosis_icd9category_token.med_terms_condensed.progress_apply(lambda x:return_feature(x))

In [ ]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_condensed,features
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[fracture, coronary, chest, hypertension, artery, cough, pain]","[f9, f33, f4, f3, f8, f37]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518, 401, 519, 348, V02, 359, 584, 458, 578, 491, E93, 276, 244]","[pulmonary, hypertension, of, obstructive, discomfort, gastrointestinal, constipation, oxygen, breath, inflammation, artery, chest, hydrochlorothiazide, steroid, anxiety, omeprazole, senna, trazodone, failure, shortness, disease, pantoprazole, aspirin, hypothyroidism, zolpidem, furosemide, coronary, prednisone, l

In [ ]:
diagnosis_icdcategory_medterms_df

,icd9category,frequent_medterms
0,401,"[pain, fever, chest, shortness, infection, breath, redness, aspirin, hypertension, atrial, fibrillation, chills, disease, coumadin, bleeding, failure, pneumonia, vomiting, fevers, constipation, swelling, nausea, abdominal, heart, diabetes, weakness, cough, anemia, erythema, headache, metoprolol, confusion, numbness, mouth, weight, diarrhea, gain, loss, lisinopril, blood, cancer, seizures, pulmonary, infarction, myocardial, bleed, stroke, motrin, artery, tylenol]"
1,427,"[pain, atrial, fibrillation, chest, shortness, fever, coumadin, infection, breath, failure, disease, heart, redness, pneumonia, aspirin, chills, hypertension, bleeding, metoprolol, fevers, renal, vomiting, anemia, abdominal, nausea, erythema, sodium, diabetes, chronic, constipation, weight, pulmonary, gain, cough, swelling, mouth, cancer, warfarin, diarrhea, lisinopril, bleed, artery, infarction, stroke, myocardial, blood, urinary, respiratory, oxygen, congestive]"
2,276,"[pain, shortness, chest, infection, breath, failure, fever, pneumonia, disease, chills, atrial, fevers, fibrillation, vomiting, bleeding, nausea, abdominal, renal, hypertension, heart, aspirin, anemia, coumadin, diarrhea, diabetes, metoprolol, mouth, chronic, constipation, cough, redness, respiratory, swelling, sodium, urinary, confusion, blood, cancer, acute, pulmonary, oxygen, tract, alcohol, lisinopril, loss, bleed, kidney, dehydration, weakness, sepsis]"
3,414,"[pain, chest, fever, shortness, infection, breath, disease, redness, atrial, failure, fibrillation, heart, aspirin, coumadin, hypertension, myocardial, infarction, erythema, bleeding, weight, gain, diabetes, chills, metoprolol, artery, renal, coronary, pneumonia, fevers, lisinopril, sodium, anemia, chronic, vomiting, mellitus, nausea, abdominal, pulmonary, constipation, edema, swelling, congestive, bleed, cancer, cholesterol, cough, mouth, blood, loss, hyperlipidemia]"
4,272,"[pain, fever, chest, infection, shortness, breath, redness, aspirin, atrial, disease, fibrillation, failure, coumadin, hypertension, bleeding, chills, heart, erythema, pneumonia, diabetes, constipation, vomiting, fevers, metoprolol, swelling, weight, renal, gain, nausea, abdominal, anemia, infarction, myocardial, cough, lisinopril, mouth, weakness, cancer, artery, confusion, mellitus, headache, pulmonary, loss, chronic, diarrhea, hyperlipidemia, blood, numbness, tylenol]"
5,250,"[pain, chest, shortness, breath, diabetes, fever, infection, failure, disease, chills, atrial, hypertension, heart, fibrillation, aspirin, fevers, mellitus, redness, pneumonia, bleeding, vomiting, renal, coumadin, nausea, abdominal, anemia, metoprolol, chronic, constipation, swelling, sodium, blood, cough, mouth, lisinopril, diarrhea, confusion, artery, erythema, pulmonary, urinary, respiratory, infarction, loss, myocardial, bleed, coronary, weakness, weight, oxygen]"
6,428,"[pain, failure, shortness, chest, heart, breath, atrial, fibrillation, disease, infection, fever, pneumonia, coumadin, chills, renal, hypertension, sodium, fevers, aspirin, metoprolol, bleeding, chronic, anemia, diabetes, congestive, redness, pulmonary, vomiting, nausea, lisinopril, abdominal, oxygen, myocardial, infarction, respiratory, artery, systolic, coronary, mouth, cough, urinary, blood, mellitus, swelling, tract, weight, bleed, diarrhea, acute, gain]"
7,518,"[pain, shortness, breath, pneumonia, failure, chest, fever, infection, disease, chills, respiratory, atrial, fibrillation, fevers, vomiting, heart, hypertension, cough, abdominal, nausea, renal, coumadin, anemia, pulmonary, redness, bleeding, aspirin, oxygen, swelling, constipation, diarrhea, mouth, chronic, cancer, diabetes, metoprolol, sodium, confusion, fracture, blood, acute, sepsis, urinary, loss, weakness, lung, prednisone, tract, obstructive, edema]"
8,285,"[pain, shortness, chest, breath, infection, fever, disease, anemia, bleeding, failure, chills, pneumonia, fevers, atrial, fibrillation, aspirin, blood, co

In [ ]:
diagnosis_icdcategory_medterms_df['feature']=diagnosis_icdcategory_medterms_df.frequent_medterms.progress_apply(lambda x:return_feature(x))

In [ ]:
diagnosis_icdcategory_medterms_df

,icd9category,frequent_medterms,feature
0,401,"[pain, fever, chest, shortness, infection, breath, redness, aspirin, hypertension, atrial, fibrillation, chills, disease, coumadin, bleeding, failure, pneumonia, vomiting, fevers, constipation, swelling, nausea, abdominal, heart, diabetes, weakness, cough, anemia, erythema, headache, metoprolol, confusion, numbness, mouth, weight, diarrhea, gain, loss, lisinopril, blood, cancer, seizures, pulmonary, infarction, myocardial, bleed, stroke, motrin, artery, tylenol]","[f29, f35, f17, f4, f11, f42, f14, f18, f26, f43, f31, f28, f6, f0, f7, f1, f27, f20, f41, f39, f8, f13, f9, f38, f33, f5, f12, f34, f37, f19]"
1,427,"[pain, atrial, fibrillation, chest, shortness, fever, coumadin, infection, breath, failure, disease, heart, redness, pneumonia, aspirin, chills, hypertension, bleeding, metoprolol, fevers, renal, vomiting, anemia, abdominal, nausea, erythema, sodium, diabetes, chronic, constipation, weight, pulmonary, gain, cough, swelling, mouth, cancer, warfarin, diarrhea, lisinopril, bleed, artery, infarction, stroke, myocardial, blood, urinary, respiratory, oxygen, congestive]","[f29, f35, f23, f17, f4, f11, f42, f14, f18, f26, f43, f28, f6, f0, f1, f21, f27, f41, f39, f45, f8, f13, f9, f22, f30, f38, f33, f15, f12, f34, f37, f19]"
2,276,"[pain, shortness, chest, infection, breath, failure, fever, pneumonia, disease, chills, atrial, fevers, fibrillation, vomiting, bleeding, nausea, abdominal, renal, hypertension, heart, aspirin, anemia, coumadin, diarrhea, diabetes, metoprolol, mouth, chronic, constipation, cough, redness, respiratory, swelling, sodium, urinary, confusion, blood, cancer, acute, pulmonary, oxygen, tract, alcohol, lisinopril, loss, bleed, kidney, dehydration, weakness, sepsis]","[f29, f35, f23, f17, f4, f11, f14, f18, f26, f25, f2, f43, f31, f28, f6, f0, f1, f21, f27, f20, f41, f39, f45, f13, f32, f9, f22, f30, f38, f33, f15, f12, f34, f37]"
3,414,"[pain, chest, fever, shortness, infection, breath, disease, redness, atrial, failure, fibrillation, heart, aspirin, coumadin, hypertension, myocardial, infarction, erythema, bleeding, weight, gain, diabetes, chills, metoprolol, artery, renal, coronary, pneumonia, fevers, lisinopril, sodium, anemia, chronic, vomiting, mellitus, nausea, abdominal, pulmonary, constipation, edema, swelling, congestive, bleed, cancer, cholesterol, cough, mouth, blood, loss, hyperlipidemia]","[f29, f36, f35, f17, f4, f11, f42, f14, f18, f26, f43, f28, f6, f0, f44, f1, f21, f27, f20, f41, f39, f45, f8, f13, f9, f22, f38, f33, f12, f34, f37, f19]"
4,272,"[pain, fever, chest, infection, shortness, breath, redness, aspirin, atrial, disease, fibrillation, failure, coumadin, hypertension, bleeding, chills, heart, erythema, pneumonia, diabetes, constipation, vomiting, fevers, metoprolol, swelling, weight, renal, gain, nausea, abdominal, anemia, infarction, myocardial, cough, lisinopril, mouth, weakness, cancer, artery, confusion, mellitus, headache, pulmonary, loss, chronic, diarrhea, hyperlipidemia, blood, numbness, tylenol]","[f29, f35, f17, f4, f11, f42, f14, f18, f26, f43, f31, f28, f6, f0, f7, f1, f21, f27, f20, f41, f39, f8, f13, f9, f22, f38, f33, f12, f34, f37, f19]"
5,250,"[pain, chest, shortness, breath, diabetes, fever, infection, failure, disease, chills, atrial, hypertension, heart, fibrillation, aspirin, fevers, mellitus, redness, pneumonia, bleeding, vomiting, renal, coumadin, nausea, abdominal, anemia, metoprolol, chronic, constipation, swelling, sodium, blood, cough, mouth, lisinopril, diarrhea, confusion, artery, erythema, pulmonary, urinary, respiratory, infarction, loss, myocardial, bleed, coronary, weakness, weight, oxygen]","[f29, f35, f23, f17, f4, f11, f42, f14, f18, f26, f31, f28, f6, f0, f1, f21, f27, f20, f41, f39, f45, f8, f13, f9, f22, f30, f38, f33, f15, f12, f34, f37]"
6,428,"[pain, failure, shortness, chest, heart, breath, atrial, fibrillation, disease, infection, fever, pneumonia, coumadin, chills, renal, hypertension, sodi

In [ ]:
def predict_icd(x):
  category_weights={}
  if len(x):
    for item1 in frequent_icd9category:
      category_features=[a for b in diagnosis_icdcategory_medterms_df.loc[diagnosis_icdcategory_medterms_df.icd9category==item1].feature.to_list() for a in b]
      count=len(list(set(x).intersection(set(category_features))))
      category_weights[item1]=count/len(x)/len(category_features)
  else:
    for item1 in frequent_icd9category:
      category_weights[item1]=0.0

      
  return category_weights
  

In [57]:
diagnosis_icd9category_token['predicted_icd_weigths']=diagnosis_icd9category_token.features.progress_apply(lambda x:predict_icd(x))

In [58]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_condensed,features,predicted_icd_weigths,predicted_icd
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[fracture, coronary, chest, hypertension, artery, cough, pain]","[f9, f33, f4, f3, f8, f37]","{'401': 0.02777777777777778, '427': 0.026041666666666668, '276': 0.0196078431372549, '414': 0.026041666666666668, '272': 0.026881720430107527, '250': 0.026041666666666668, '428': 0.025252525252525252, '518': 0.02314814814814815, '285': 0.02380952380952381, '584': 0.020833333333333332}","[401, 272, 427, 414, 250, 428, 285, 518, 584, 276]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518,

In [59]:
def predictbasedonwts(x):
  x1=dict(sorted(x.items(),key= lambda y: y[1],reverse=True))
  x2={k:v for (k,v) in x1.items() if v>0}
  repeatedval=dict(Counter(list(x2.values())))
  repeatedvalgreaterthan1=[k for (k,v) in repeatedval.items() if v>1]
  x3=dict(list(x2.items())[:10])
  x4=[v for (k,v) in x3.items() if v in repeatedvalgreaterthan1]
  x5={k:v for (k,v) in x2.items() if v in x4}
  x3.update(x5)
  return list(x3.keys())

In [60]:
diagnosis_icd9category_token['predicted_icd']=diagnosis_icd9category_token.predicted_icd_weigths.progress_apply(predictbasedonwts)

In [61]:
diagnosis_icd9category_token

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_condensed,features,predicted_icd_weigths,predicted_icd
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[fracture, coronary, chest, hypertension, artery, cough, pain]","[f9, f33, f4, f3, f8, f37]","{'401': 0.02777777777777778, '427': 0.026041666666666668, '276': 0.0196078431372549, '414': 0.026041666666666668, '272': 0.026881720430107527, '250': 0.026041666666666668, '428': 0.025252525252525252, '518': 0.02314814814814815, '285': 0.02380952380952381, '584': 0.020833333333333332}","[401, 272, 427, 414, 250, 428, 285, 518, 584, 276]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518,

In [62]:
result_predict={'category':[],'precision':[],'recall':[],'f1':[]}
for i in frequent_icd9category:
  actual_p=diagnosis_icd9category_token.loc[diagnosis_icd9category_token.icd9_category.apply(lambda x:i in x)].HADM_ID.count()
  tp=diagnosis_icd9category_token.loc[(diagnosis_icd9category_token.icd9_category.apply(lambda x:i in x))&(diagnosis_icd9category_token.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
  predict_p=diagnosis_icd9category_token.loc[diagnosis_icd9category_token.predicted_icd.apply(lambda x:i in x)].HADM_ID.count()
  precision=tp/predict_p
  recall=tp/actual_p
  f1=2*precision*recall/(precision+recall)
  result_predict['category'].append(i)
  result_predict['precision'].append(precision)
  result_predict['recall'].append(recall)
  result_predict['f1'].append(f1)

In [63]:
pd.DataFrame.from_dict(result_predict)

,category,precision,recall,f1
0,401,0.432877,0.854584,0.574665
1,427,0.346660,0.853080,0.492988
2,276,0.312711,0.837563,0.455396
3,414,0.292055,0.874852,0.437918
4,272,0.320960,0.879334,0.470270
5,250,0.296993,0.866474,0.442362
6,428,0.279617,0.870110,0.423227
7,518,0.256174,0.807617,0.388969
8,285,0.291350,0.892718,0.439322
9,584,0.239515,0.834991,0.372251


In [64]:
total_predicted=len([a for b in diagnosis_icd9category_token.predicted_icd.to_list() for a in b])
print(total_predicted)
actual=len([a for b in diagnosis_icd9category_token.icd9_category.apply(lambda x : [i for i in x if i in frequent_icd9category]).to_list() for a in b])
print(actual)
actual_predicted=0
for i in frequent_icd9category:
  actual_predicted += diagnosis_icd9category_token.loc[(diagnosis_icd9category_token.icd9_category.apply(lambda x:i in x))&(diagnosis_icd9category_token.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
print(actual_predicted)
recall=actual_predicted/actual
presicion=actual_predicted/total_predicted
print(recall)
print(presicion)
print(2*presicion*recall/(presicion + recall))

352853
126319
108297
0.8573294595429033
0.30691817839156815
0.4520172297212692


In [ ]:
mlb = MultiLabelBinarizer(frequent_icd9category)
print(accuracy_score(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(diagnosis_icd9category_token.predicted_icd.values)))
print(hamming_loss(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(diagnosis_icd9category_token.predicted_icd.values)))
print(precision_score(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(diagnosis_icd9category_token.predicted_icd.values), average = "micro"))
print(recall_score(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(diagnosis_icd9category_token.predicted_icd.values),average = "micro"))
print(f1_score(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(ddiagnosis_icd9category_token.predicted_icd.values),average = "micro"))
print(classification_report(mlb.fit_transform(diagnosis_icd9category_token.icd9_category.values),mlb.fit_transform(diagnosis_icd9category_token.predicted_icd.values),target_names=mlb.classes_))

In [66]:
adm_icd = pd.read_csv("/content/drive/My Drive/AlgoIntern/Data/Data1.0/adm_icd.csv")

In [67]:
diagnosis_icd10category=pd.merge(diagnosis_icd9category_token['HADM_ID','discharge diagnosis:', 'medical_terms_list'],adm_icd[['HADM_ID','ICD10']],on="HADM_ID",how="left")

In [68]:
diagnosis_icd10category

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_condensed,features,predicted_icd_weigths,predicted_icd,ICD10
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[fracture, coronary, chest, hypertension, artery, cough, pain]","[f9, f33, f4, f3, f8, f37]","{'401': 0.02777777777777778, '427': 0.026041666666666668, '276': 0.0196078431372549, '414': 0.026041666666666668, '272': 0.026881720430107527, '250': 0.026041666666666668, '428': 0.025252525252525252, '518': 0.02314814814814815, '285': 0.02380952380952381, '584': 0.020833333333333332}","[401, 272, 427, 414, 250, 428, 285, 518, 584, 276]","J449,Misc,E872"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 5

In [69]:
diagnosis_icd10category['ICD10']=diagnosis_icd10category.ICD10.apply(lambda x:list(set(re.split(",",x))))

In [71]:
def icd10codes_into_category(x):
    return list(set([re.split("([A-Z]\d{2})",i)[1] for i in x]))

In [72]:
diagnosis_icd10category['ICD10']=diagnosis_icd10category.ICD10.apply(lambda x: [a for a in x if a!="Misc"])

In [73]:
diagnosis_icd10category['icd10category']=diagnosis_icd10category.ICD10.apply(icd10codes_into_category)

In [74]:
diagnosis_icd10category

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_condensed,features,predicted_icd_weigths,predicted_icd,ICD10,icd10category
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","[hypercholesterolemia, cerebral, hypothyroidism, cough, accident, hypertension, artery, chest, vascular, coronary, fracture, injury, angina, pain, head]","[fracture, coronary, chest, hypertension, artery, cough, pain]","[f9, f33, f4, f3, f8, f37]","{'401': 0.02777777777777778, '427': 0.026041666666666668, '276': 0.0196078431372549, '414': 0.026041666666666668, '272': 0.026881720430107527, '250': 0.026041666666666668, '428': 0.025252525252525252, '518': 0.02314814814814815, '285': 0.02380952380952381, '584': 0.020833333333333332}","[401, 272, 427, 414, 250, 428, 285, 518, 584, 276]","[E872, J449]","[E87, J44]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main g

In [75]:
frequent_icd10category = ['I10', 'I25', 'E78', 'I50', 'I48', 'N17', 'E87', 'E11', 'J96', 'N39']

In [76]:
category_med_terms={}
common_diagnosis_terms=["monday","telephone","of","phone"]
for i in frequent_icd10category:
    diagnosis_x=diagnosis_icd10category.loc[diagnosis_icd10category.icd10category.apply(lambda x: i in x)].copy()
    y=diagnosis_x.medical_terms_list.to_list()    
    z=[list(set(a)) for a in y]
    z1=[a for b in z for a in b]
    dict_med=dict(Counter(z1))
    dict_med=dict(sorted(dict_med.items(),key= lambda x: x[1],reverse=True))
    dict_med_list=list(dict_med)
    dict_med_list1=[a for a in dict_med_list if a not in common_diagnosis_terms]
    category_med_terms[i]=dict_med_list1[:50]

diagnosis_icdcategory_medterms={'icd10category':[i for i in category_med_terms],
                                'frequent_medterms':[category_med_terms[i] for i in category_med_terms]}
diagnosis_icdcategory_medterms_df=pd.DataFrame.from_dict(diagnosis_icdcategory_medterms)

diagnosis_icdcategory_medterms_df

,icd10category,frequent_medterms
0,I10,"[pain, fever, chest, shortness, infection, breath, redness, aspirin, hypertension, atrial, fibrillation, chills, disease, coumadin, bleeding, failure, pneumonia, vomiting, fevers, constipation, swelling, nausea, abdominal, heart, diabetes, erythema, cough, weakness, anemia, metoprolol, headache, confusion, mouth, numbness, weight, diarrhea, gain, lisinopril, loss, cancer, blood, infarction, seizures, pulmonary, myocardial, stroke, bleed, motrin, artery, tylenol]"
1,I25,"[pain, chest, fever, shortness, infection, breath, disease, redness, atrial, failure, fibrillation, aspirin, heart, coumadin, hypertension, myocardial, infarction, erythema, bleeding, weight, chills, gain, diabetes, metoprolol, artery, renal, coronary, pneumonia, fevers, lisinopril, anemia, vomiting, sodium, chronic, nausea, mellitus, abdominal, constipation, pulmonary, swelling, edema, cough, mouth, congestive, cholesterol, bleed, cancer, blood, loss, diarrhea]"
2,E78,"[pain, fever, chest, infection, shortness, breath, redness, aspirin, atrial, disease, fibrillation, failure, coumadin, hypertension, bleeding, chills, heart, erythema, pneumonia, diabetes, constipation, vomiting, fevers, metoprolol, swelling, weight, renal, gain, nausea, anemia, abdominal, infarction, myocardial, lisinopril, cough, mouth, weakness, cancer, artery, confusion, mellitus, headache, pulmonary, loss, chronic, hyperlipidemia, diarrhea, blood, numbness, tylenol]"
3,I50,"[pain, failure, shortness, chest, heart, breath, atrial, fibrillation, disease, infection, fever, pneumonia, coumadin, chills, renal, hypertension, fevers, sodium, aspirin, metoprolol, bleeding, chronic, anemia, congestive, diabetes, redness, pulmonary, vomiting, nausea, lisinopril, abdominal, oxygen, myocardial, infarction, respiratory, artery, systolic, coronary, mouth, urinary, cough, blood, mellitus, swelling, tract, bleed, weight, acute, diarrhea, gain]"
4,I48,"[atrial, pain, fibrillation, coumadin, shortness, chest, fever, failure, infection, breath, disease, heart, redness, pneumonia, chills, aspirin, hypertension, bleeding, metoprolol, renal, fevers, anemia, erythema, vomiting, abdominal, sodium, weight, gain, nausea, chronic, diabetes, pulmonary, warfarin, constipation, bleed, stroke, cough, swelling, cancer, mouth, diarrhea, artery, urinary, congestive, blood, tract, respiratory, oxygen, lisinopril, amiodarone]"
5,N17,"[pain, failure, shortness, chest, infection, breath, renal, disease, pneumonia, fever, atrial, chills, fibrillation, heart, fevers, chronic, hypertension, vomiting, bleeding, acute, abdominal, coumadin, anemia, nausea, diabetes, aspirin, metoprolol, kidney, diarrhea, sodium, respiratory, mouth, urinary, lisinopril, tract, blood, pulmonary, cough, cancer, redness, sepsis, swelling, oxygen, constipation, bleed, loss, confusion, congestive, mellitus, dizziness]"
6,E87,"[pain, infection, shortness, failure, chest, breath, pneumonia, fever, disease, chills, atrial, fibrillation, fevers, renal, bleeding, vomiting, hypertension, abdominal, heart, nausea, aspirin, anemia, coumadin, chronic, diabetes, diarrhea, metoprolol, respiratory, mouth, sodium, constipation, redness, confusion, urinary, swelling, acute, cough, tract, alcohol, blood, pulmonary, cancer, lisinopril, oxygen, kidney, loss, bleed, sepsis, weakness, seizures]"
7,E11,"[pain, shortness, chest, fever, breath, infection, failure, diabetes, disease, chills, atrial, fibrillation, redness, heart, aspirin, hypertension, bleeding, coumadin, fevers, pneumonia, mellitus, vomiting, renal, nausea, abdominal, metoprolol, anemia, constipation, swelling, chronic, sodium, cough, erythema, mouth, lisinopril, diarrhea, blood, confusion, artery, weight, pulmonary, infarction, gain, bleed, weakness, myocardial, respiratory, oxygen, urinary, loss]"
8,J96,"[pain, pneumonia, failure, shortness, breath, chest, respiratory, infection, disease, fever, chills, atrial, fibrillation, fevers, heart, hypertension, renal, vomiting, 

In [ ]:
bagofwords=list(set([a for b in diagnosis_icdcategory_medterms_df.frequent_medterms.to_list() for a in b]))

In [ ]:
simi_matrix=np.zeros((len(bagofwords),len(bagofwords)))

In [ ]:
simi_matrix.shape

In [ ]:
for i in tqdm(range(len(bagofwords))):
  for j in range(len(bagofwords)):
    simi_matrix[i][j]=model.similarity(bagofwords[i],bagofwords[j])

In [ ]:
features={}
for i in range(len(bagofwords)-1):
  similar=[]
  for j in range(i+1,len(bagofwords)):
    if simi_matrix[i][j]>=0.7:
      similar.append(bagofwords[j])
  features[bagofwords[i]]=similar
features[bagofwords[-1]]=[]    


In [ ]:
len(features)

In [ ]:
features_list=[features[k]+[k] for k in list(features)]

In [ ]:
condensed_feature = list(merge_common(features_list)) 

In [ ]:
condensed_feature_dict={}
count=0
for i in condensed_feature:
  condensed_feature_dict['f'+str(count)]=i
  count+=1

In [ ]:
all_vocab=[a for b in condensed_feature for a in b]

In [ ]:
diagnosis_icd10category['med_terms_condensed']=diagnosis_icd10category.medical_terms_list.progress_apply(lambda x:list(set(x).intersection(set(all_vocab))))
diagnosis_icd10category['features']=diagnosis_icd10category.med_terms_condensed.progress_apply(lambda x:return_feature(x))

In [ ]:
diagnosis_icdcategory_medterms_df['feature']=diagnosis_icdcategory_medterms_df.frequent_medterms.progress_apply(lambda x:return_feature(x))

In [ ]:
def predict_icd(x):
  category_weights={}
  if len(x):
    for item1 in frequent_icd10category:
      category_features=[a for b in diagnosis_icdcategory_medterms_df.loc[diagnosis_icdcategory_medterms_df.icd10category==item1].feature.to_list() for a in b]
      count=len(list(set(x).intersection(set(category_features))))
      category_weights[item1]=count/len(x)/len(category_features)
  else:
    for item1 in frequent_icd10category:
      category_weights[item1]=0.0

      
  return category_weights
  

In [ ]:
diagnosis_icd10category['predicted_icd_weigths']=diagnosis_icd10category.features.progress_apply(lambda x:predict_icd(x))
diagnosis_icd10category['predicted_icd']=diagnosis_icd10category.predicted_icd_weigths.progress_apply(predictbasedonwts)

In [ ]:
result_predict={'category':[],'precision':[],'recall':[],'f1':[]}
for i in frequent_icd10category:
  actual_p=diagnosis_icd10category.loc[diagnosis_icd10category.icd10category.apply(lambda x:i in x)].HADM_ID.count()
  tp=diagnosis_icd10category.loc[(diagnosis_icd10category.icd10category.apply(lambda x:i in x))&(diagnosis_icd10category.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
  predict_p=diagnosis_icd10category.loc[diagnosis_icd10category.predicted_icd.apply(lambda x:i in x)].HADM_ID.count()
  precision=tp/predict_p
  recall=tp/actual_p
  f1=2*precision*recall/(precision+recall)
  result_predict['category'].append(i)
  result_predict['precision'].append(precision)
  result_predict['recall'].append(recall)
  result_predict['f1'].append(f1)

In [ ]:
pd.DataFrame.from_dict(result_predict)

In [ ]:
total_predicted=len([a for b in diagnosis_icd10category.predicted_icd.to_list() for a in b])
print(total_predicted)
actual=len([a for b in diagnosis_icd10category.icd10category.apply(lambda x : [i for i in x if i in frequent_icd10category]).to_list() for a in b])
print(actual)
actual_predicted=0
for i in frequent_icd10category:
  actual_predicted += diagnosis_icd10category.loc[(diagnosis_icd10category.icd10category.apply(lambda x:i in x))&(diagnosis_icd10category.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
print(actual_predicted)
recall=actual_predicted/actual
presicion=actual_predicted/total_predicted
print(recall)
print(presicion)
print(2*presicion*recall/(presicion + recall))

In [ ]:
mlb = MultiLabelBinarizer(frequent_icd10category)
print(accuracy_score(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values)))
print(hamming_loss(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values)))
print(precision_score(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values), average = "micro"))
print(recall_score(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values),average = "micro"))
print(f1_score(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values),average = "micro"))
print(classification_report(mlb.fit_transform(diagnosis_icd10category.icd10category.values),mlb.fit_transform(diagnosis_icd10category.predicted_icd.values),target_names=mlb.classes_))

In [ ]:
diagnosis_term_codes=pd.merge(diagnosis_icd9category_token[['HADM_ID','discharge diagnosis:', 'medical_terms_list']],adm_icd,on="HADM_ID",how="left")

In [ ]:
diagnosis_terms_codes['ICD9_CODE']=diagnosis_terms_codes.ICD9_CODE.apply(lambda x:list(set(re.split(",",x))))
diagnosis_terms_codes['ICD10']=diagnosis_terms_codes.ICD10.apply(lambda x:list(set(re.split(",",x))))

In [ ]:
diagnosis_terms_codes['ICD10']=diagnosis_terms_codes.ICD10.apply(lambda x:[a for a in x if a!="Misc"])

In [ ]:
frequent_icd9codes = ['4019', '4280', '42731', '41401', '5849', '25000', '2724', '51881', '5990', '53081']

In [ ]:
code_med_terms={}
common_diagnosis_terms=["monday","telephone","of","phone"]
for i in frequent_icd9codes:
    diagnosis_x=diagnosis_terms_codes.loc[diagnosis_terms_codes.ICD9_CODE.apply(lambda x: i in x)].copy()
    y=diagnosis_x.medical_terms_list.to_list()    
    z=[list(set(a)) for a in y]
    z1=[a for b in z for a in b]
    dict_med=dict(Counter(z1))
    dict_med=dict(sorted(dict_med.items(),key= lambda x: x[1],reverse=True))
    dict_med_list=list(dict_med)
    dict_med_list1=[a for a in dict_med_list if a not in common_diagnosis_terms]
    code_med_terms[i]=dict_med_list1[:50]

diagnosis_icdcategory_medterms={'icd9code':[i for i in code_med_terms],
                                'frequent_medterms':[code_med_terms[i] for i in code_med_terms]}
diagnosis_icdcategory_medterms_df=pd.DataFrame.from_dict(diagnosis_icdcategory_medterms)

display(diagnosis_icdcategory_medterms_df)

bagofwords=list(set([a for b in diagnosis_icdcategory_medterms_df.frequent_medterms.to_list() for a in b]))
simi_matrix=np.zeros((len(bagofwords),len(bagofwords)))

for i in tqdm(range(len(bagofwords))):
  for j in range(len(bagofwords)):
    simi_matrix[i][j]=model.similarity(bagofwords[i],bagofwords[j])

features={}
for i in range(len(bagofwords)-1):
  similar=[]
  for j in range(i+1,len(bagofwords)):
    if simi_matrix[i][j]>=0.7:
      similar.append(bagofwords[j])
  features[bagofwords[i]]=similar
features[bagofwords[-1]]=[]    

features_list=[features[k]+[k] for k in list(features)]

condensed_feature = list(merge_common(features_list)) 

condensed_feature_dict={}
count=0
for i in condensed_feature:
  condensed_feature_dict['f'+str(count)]=i
  count+=1

all_vocab=[a for b in condensed_feature for a in b]

diagnosis_terms_codes['med_terms_condensed']=diagnosis_terms_codes.medical_terms_list.progress_apply(lambda x:list(set(x).intersection(set(all_vocab))))
diagnosis_terms_codes['features']=diagnosis_terms_codes.med_terms_condensed.progress_apply(lambda x:return_feature(x))

diagnosis_icdcategory_medterms_df['feature']=diagnosis_icdcategory_medterms_df.frequent_medterms.progress_apply(lambda x:return_feature(x))


def predict_icd(x):
  category_weights={}
  if len(x):
    for item1 in frequent_icd9codes:
      category_features=[a for b in diagnosis_icdcategory_medterms_df.loc[diagnosis_icdcategory_medterms_df.icd9code==item1].feature.to_list() for a in b]
      count=len(list(set(x).intersection(set(category_features))))
      category_weights[item1]=count/len(x)/len(category_features)
  else:
    for item1 in frequent_icd9codes:
      category_weights[item1]=0.0

      
  return category_weights



diagnosis_terms_codes['predicted_icd_weigths']=diagnosis_terms_codes.features.progress_apply(lambda x:predict_icd(x))
diagnosis_terms_codes['predicted_icd']=diagnosis_terms_codes.predicted_icd_weigths.progress_apply(predictbasedonwts)


result_predict={'category':[],'precision':[],'recall':[],'f1':[]}
for i in frequent_icd9codes:
  actual_p=diagnosis_terms_codes.loc[diagnosis_terms_codes.ICD9_CODE.apply(lambda x:i in x)].HADM_ID.count()
  tp=diagnosis_terms_codes.loc[(diagnosis_terms_codes.ICD9_CODE.apply(lambda x:i in x))&(diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
  predict_p=diagnosis_terms_codes.loc[diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x)].HADM_ID.count()
  precision=tp/predict_p
  recall=tp/actual_p
  f1=2*precision*recall/(precision+recall)
  result_predict['category'].append(i)
  result_predict['precision'].append(precision)
  result_predict['recall'].append(recall)
  result_predict['f1'].append(f1)



display(pd.DataFrame.from_dict(result_predict))


total_predicted=len([a for b in diagnosis_terms_codes.predicted_icd.to_list() for a in b])
print(total_predicted)
actual=len([a for b in diagnosis_terms_codes.ICD9_CODE.apply(lambda x : [i for i in x if i in frequent_icd9codes]).to_list() for a in b])
print(actual)
actual_predicted=0
for i in frequent_icd9codes:
  actual_predicted += diagnosis_terms_codes.loc[(diagnosis_terms_codes.ICD9_CODE.apply(lambda x:i in x))&(diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
print(actual_predicted)
recall=actual_predicted/actual
presicion=actual_predicted/total_predicted
print(recall)
print(presicion)
print(2*presicion*recall/(presicion + recall))


In [ ]:
mlb = MultiLabelBinarizer(frequent_icd9codes)
print(accuracy_score(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values)))
print(hamming_loss(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values)))
print(precision_score(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values), average = "micro"))
print(recall_score(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),average = "micro"))
print(f1_score(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),average = "micro"))
print(classification_report(mlb.fit_transform(diagnosis_terms_codes.ICD9_CODE.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),target_names=mlb.classes_))

In [ ]:
frequent_icd10codes = ['I10', 'I2510', 'I509', 'I4891', 'N179', 'E119', 'E784', 'E785', 'J9690', 'J9600']

In [ ]:
code_med_terms={}
common_diagnosis_terms=["monday","telephone","of","phone"]
for i in frequent_icd10codes:
    diagnosis_x=diagnosis_terms_codes.loc[diagnosis_terms_codes.ICD10.apply(lambda x: i in x)].copy()
    y=diagnosis_x.medical_terms_list.to_list()    
    z=[list(set(a)) for a in y]
    z1=[a for b in z for a in b]
    dict_med=dict(Counter(z1))
    dict_med=dict(sorted(dict_med.items(),key= lambda x: x[1],reverse=True))
    dict_med_list=list(dict_med)
    dict_med_list1=[a for a in dict_med_list if a not in common_diagnosis_terms]
    code_med_terms[i]=dict_med_list1[:50]

diagnosis_icdcategory_medterms={'icd10code':[i for i in code_med_terms],
                                'frequent_medterms':[code_med_terms[i] for i in code_med_terms]}
diagnosis_icdcategory_medterms_df=pd.DataFrame.from_dict(diagnosis_icdcategory_medterms)

display(diagnosis_icdcategory_medterms_df)

bagofwords=list(set([a for b in diagnosis_icdcategory_medterms_df.frequent_medterms.to_list() for a in b]))
simi_matrix=np.zeros((len(bagofwords),len(bagofwords)))

for i in tqdm(range(len(bagofwords))):
  for j in range(len(bagofwords)):
    simi_matrix[i][j]=model.similarity(bagofwords[i],bagofwords[j])

features={}
for i in range(len(bagofwords)-1):
  similar=[]
  for j in range(i+1,len(bagofwords)):
    if simi_matrix[i][j]>=0.7:
      similar.append(bagofwords[j])
  features[bagofwords[i]]=similar
features[bagofwords[-1]]=[]    

features_list=[features[k]+[k] for k in list(features)]

condensed_feature = list(merge_common(features_list)) 

condensed_feature_dict={}
count=0
for i in condensed_feature:
  condensed_feature_dict['f'+str(count)]=i
  count+=1

all_vocab=[a for b in condensed_feature for a in b]

diagnosis_terms_codes['med_terms_condensed']=diagnosis_terms_codes.medical_terms_list.progress_apply(lambda x:list(set(x).intersection(set(all_vocab))))
diagnosis_terms_codes['features']=diagnosis_terms_codes.med_terms_condensed.progress_apply(lambda x:return_feature(x))

diagnosis_icdcategory_medterms_df['feature']=diagnosis_icdcategory_medterms_df.frequent_medterms.progress_apply(lambda x:return_feature(x))


def predict_icd(x):
  category_weights={}
  if len(x):
    for item1 in frequent_icd10codes:
      category_features=[a for b in diagnosis_icdcategory_medterms_df.loc[diagnosis_icdcategory_medterms_df.icd10code==item1].feature.to_list() for a in b]
      count=len(list(set(x).intersection(set(category_features))))
      category_weights[item1]=count/len(x)/len(category_features)
  else:
    for item1 in frequent_icd10codes:
      category_weights[item1]=0.0

      
  return category_weights



diagnosis_terms_codes['predicted_icd_weigths']=diagnosis_terms_codes.features.progress_apply(lambda x:predict_icd(x))
diagnosis_terms_codes['predicted_icd']=diagnosis_terms_codes.predicted_icd_weigths.progress_apply(predictbasedonwts)


result_predict={'category':[],'precision':[],'recall':[],'f1':[]}
for i in frequent_icd10codes:
  actual_p=diagnosis_terms_codes.loc[diagnosis_terms_codes.ICD10.apply(lambda x:i in x)].HADM_ID.count()
  tp=diagnosis_terms_codes.loc[(diagnosis_terms_codes.ICD10.apply(lambda x:i in x))&(diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
  predict_p=diagnosis_terms_codes.loc[diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x)].HADM_ID.count()
  precision=tp/predict_p
  recall=tp/actual_p
  f1=2*precision*recall/(precision+recall)
  result_predict['category'].append(i)
  result_predict['precision'].append(precision)
  result_predict['recall'].append(recall)
  result_predict['f1'].append(f1)



display(pd.DataFrame.from_dict(result_predict))


total_predicted=len([a for b in diagnosis_terms_codes.predicted_icd.to_list() for a in b])
print(total_predicted)
actual=len([a for b in diagnosis_terms_codes.ICD10.apply(lambda x : [i for i in x if i in frequent_icd10codes]).to_list() for a in b])
print(actual)
actual_predicted=0
for i in frequent_icd10codes:
  actual_predicted += diagnosis_terms_codes.loc[(diagnosis_terms_codes.ICD10.apply(lambda x:i in x))&(diagnosis_terms_codes.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
print(actual_predicted)
recall=actual_predicted/actual
presicion=actual_predicted/total_predicted
print(recall)
print(presicion)
print(2*presicion*recall/(presicion + recall))


In [ ]:
mlb = MultiLabelBinarizer(frequent_icd10codes)
print(accuracy_score(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values)))
print(hamming_loss(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values)))
print(precision_score(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values), average = "micro"))
print(recall_score(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),average = "micro"))
print(f1_score(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),average = "micro"))
print(classification_report(mlb.fit_transform(diagnosis_terms_codes.ICD10.values),mlb.fit_transform(diagnosis_terms_codes.predicted_icd.values),target_names=mlb.classes_))

In [ ]:
# diagnosis_icd9category=pd.read_csv("/content/drive/My Drive/AlgoIntern/Data/Data2.0/diagnosis_icd9category.csv")
# diagnosis_icd9category['icd9_category']=diagnosis_icd9category.icd9_category.apply(literal_eval)
# diagnosis_icd9category.dropna(subset=['discharge diagnosis:'],inplace=True)
# diagnosis_icd9category.reset_index(drop=True,inplace=True)

In [ ]:
# ! pip install scispacy
# ! pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_md-0.2.5.tar.gz

In [ ]:
# import spacy
# import scispacy
# import en_core_sci_md

In [ ]:
# nlp=en_core_sci_md.load()

In [ ]:
# diagnosis_icd9category['medical_terms_list']=diagnosis_icd9category['discharge diagnosis:'].progress_apply(lambda x: nlp(x).ents)
# diagnosis_icd9category['medical_terms_list']=diagnosis_icd9category.medical_terms_list.progress_apply(lambda x: [i.text for a in x for i in a])
# diagnosis_icd9category['medical_terms_list']=diagnosis_icd9category.medical_terms_list.apply(lambda x:list(set([str(i) for i in x])))
# diagnosis_icd9category['med_terms_token']=diagnosis_icd9category.medical_terms_list.progress_apply(lambda x: [word_tokenize(i) for i in x])
# diagnosis_icd9category['med_terms_token']=diagnosis_icd9category.med_terms_token.apply(lambda x:list(set([a for b in x for a in b if a not in list(string.punctuation)])))

In [ ]:
# diagnosis_icd9category.to_csv("/content/drive/My Drive/AlgoIntern/Data/Data2.0/diagnosis_icd9category_tokens_scimd.csv",index=False)

In [ ]:
diagnosis_icd9category=pd.read_csv("/content/drive/My Drive/AlgoIntern/Data/Data2.0/diagnosis_icd9category_tokens_scimd.csv")

In [ ]:
diagnosis_icd9category['icd9_category']=diagnosis_icd9category.icd9_category.apply(literal_eval)
diagnosis_icd9category['med_terms_token']=diagnosis_icd9category.med_terms_token.apply(literal_eval)

In [ ]:
diagnosis_icd9category

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_token
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","['discharge diagnosis', 'coronary artery disease/atypical angina', 'hypothyroidism', 'rca', 'cerebral vascular accident', 'questions', 'hypertension', 'discharge condition', 'productive cough', 'concerns', 'chest pain', 'hypercholesterolemia', 'discharge', 'hiatal hernia', 'rib fracture', 's/p motor vehicle', 'office', 'telephone']","[fracture, pain, rib, s/p, discharge, artery, accident, diagnosis, rca, hernia, productive, concerns, cerebral, hiatal, vehicle, telephone, questions, hypothyroidism, hypertension, angina, condition, disease/atypical, office, chest, coronary, hypercholesterolemia, vascular, motor, cough]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parking: main garage","[487, 416, 272, 530, 414, 553, 531, 518, 401, 519, 3

In [ ]:
category_med_terms={}
common_diagnosis_terms=["monday","telephone","of","phone",'diagnosis', 'discharge', 'condition', 'followup', 'your', 'instructions','weeks', 'medications', 'appointment', 'hospital', 'care', 'md','admitted', 'office','primary', 'follow-up', 'day', 'appointments', 'prescribed', 'coherent', 'week', 'name', 'ambulatory', 'home', '**location', 'daily','days', 'increased', 'clinic', 'changes','medication','physician','no','doctor','patient','parking','scheduled','schedule','question','treated','independent','month','morning','garage']
for i in frequent_icd9category:
    diagnosis_x=diagnosis_icd9category.loc[diagnosis_icd9category.icd9_category.apply(lambda x: i in x)].copy()
    y=diagnosis_x.med_terms_token.to_list()    
    z=[list(set(a)) for a in y]
    z1=[a for b in z for a in b]
    dict_med=dict(Counter(z1))
    dict_med=dict(sorted(dict_med.items(),key= lambda x: x[1],reverse=True))
    dict_med_list=list(dict_med)
    dict_med_list1=[a for a in dict_med_list if a not in common_diagnosis_terms]
    category_med_terms[i]=dict_med_list1[:30]

In [ ]:
diagnosis_icdcategory_medterms={'icd9category':[i for i in category_med_terms],
                                'frequent_medterms':[category_med_terms[i] for i in category_med_terms]}
diagnosis_icdcategory_medterms_df=pd.DataFrame.from_dict(diagnosis_icdcategory_medterms)

In [ ]:
diagnosis_icdcategory_medterms_df

,icd9category,frequent_medterms
0,401,"[pain, blood, consciousness, drainage, alert, chest, disease, interactive, fever, redness, surgery, infection, artery, shortness, symptoms, heart, incision, incisions, coronary, breath, s/p, baths, questions, left, pcp, right, hours, swimming, surgeon, aspirin]"
1,427,"[pain, atrial, fibrillation, heart, blood, disease, chest, consciousness, failure, alert, interactive, drainage, infection, shortness, coumadin, artery, fever, chronic, surgery, coronary, symptoms, cardiac, breath, s/p, redness, incisions, baths, questions, incision, left]"
2,276,"[pain, blood, consciousness, alert, interactive, chest, failure, disease, symptoms, infection, shortness, heart, antibiotics, breath, chronic, fever, acute, pressure, renal, pcp, chills, pneumonia, secondary, surgery, abdominal, atrial, low, medical, emergency, department]"
3,272,"[pain, blood, disease, consciousness, alert, drainage, interactive, artery, chest, coronary, heart, surgery, redness, incisions, fever, infection, incision, cardiac, baths, questions, s/p, shortness, swimming, hours, symptoms, breath, failure, left, lotions, chronic]"
4,250,"[pain, blood, disease, consciousness, chest, alert, interactive, heart, failure, shortness, diabetes, infection, artery, symptoms, fever, chronic, breath, drainage, coronary, surgery, redness, renal, pressure, antibiotics, pcp, s/p, chills, cardiac, left, incisions]"
5,414,"[pain, disease, artery, coronary, heart, drainage, blood, chest, incisions, cardiac, redness, baths, fever, surgery, s/p, consciousness, questions, failure, lotions, alert, swimming, infection, hours, interactive, shortness, chronic, incision, ointments, breath, atrial]"
6,285,"[pain, blood, consciousness, alert, interactive, chest, disease, shortness, infection, symptoms, failure, heart, surgery, breath, fever, chronic, drainage, antibiotics, bleeding, pcp, acute, pressure, chills, redness, emergency, renal, incision, artery, secondary, medical]"
7,428,"[heart, pain, failure, disease, blood, chest, chronic, shortness, consciousness, alert, breath, interactive, atrial, fibrillation, artery, infection, congestive, symptoms, coronary, cardiac, fever, renal, acute, antibiotics, coumadin, surgery, pressure, diet, s/p, pcp]"
8,518,"[pain, failure, chest, blood, consciousness, shortness, alert, interactive, disease, breath, pneumonia, symptoms, heart, infection, fever, respiratory, antibiotics, chronic, chills, surgery, breathing, acute, drainage, pcp, pulmonary, atrial, right, fibrillation, renal, emergency]"
9,584,"[pain, failure, blood, consciousness, alert, renal, interactive, heart, chest, acute, disease, chronic, infection, shortness, symptoms, antibiotics, breath, pressure, kidney, fever, pneumonia, pcp, atrial, chills, secondary, fibrillation, low, surgery, abdominal, medical]"


In [ ]:
y=diagnosis_icdcategory_medterms_df.frequent_medterms.to_list()
y1=[a for b in y for a in b]
medical=dict(Counter(y1))
print([k for (k,v) in medical.items() if v>=16])

['pain', 'blood', 'consciousness', 'alert', 'chest', 'disease', 'interactive', 'fever', 'surgery', 'infection', 'shortness', 'symptoms', 'heart', 'breath', 'pcp', 'failure', 'chronic']


In [ ]:
bagofwords=list(set([a for b in diagnosis_icdcategory_medterms_df.frequent_medterms.to_list() for a in b]))

In [ ]:
len(bagofwords)

66

In [ ]:
set(bagofwords).issubset(model.vocab)

True

In [ ]:
simi_matrix=np.zeros((len(bagofwords),len(bagofwords)))

In [ ]:
for i in tqdm(range(len(bagofwords))):
  for j in range(len(bagofwords)):
    simi_matrix[i][j]=model.similarity(bagofwords[i],bagofwords[j])


  0%|          | 0/66 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):

100%|██████████| 66/66 [00:00<00:00, 381.19it/s]


In [ ]:
simi_matrix

array([[1.        , 0.36373129, 0.44717848, ..., 0.4776668 , 0.27969456,
        0.5930348 ],
       [0.36373129, 1.00000012, 0.64902401, ..., 0.30236477, 0.2982412 ,
        0.36406419],
       [0.44717848, 0.64902401, 0.99999994, ..., 0.41321895, 0.26900902,
        0.36631155],
       ...,
       [0.4776668 , 0.30236477, 0.41321895, ..., 1.        , 0.32790142,
        0.28980109],
       [0.27969456, 0.2982412 , 0.26900902, ..., 0.32790142, 1.        ,
        0.2268275 ],
       [0.5930348 , 0.36406419, 0.36631155, ..., 0.28980109, 0.2268275 ,
        1.        ]])

In [ ]:
features={}
for i in range(len(bagofwords)-1):
  similar=[]
  for j in range(i+1,len(bagofwords)):
    if simi_matrix[i][j]>=0.7:
      similar.append(bagofwords[j])
  features[bagofwords[i]]=similar
features[bagofwords[-1]]=[]    


In [ ]:
features_list=[features[k]+[k] for k in list(features)]

In [ ]:
len(features_list)

66

In [ ]:
condensed_feature = list(merge_common(features_list))

In [ ]:
condensed_feature

[['alert'],
 ['medical'],
 ['department', 'emergency'],
 ['cardiac', 'heart'],
 ['pain'],
 ['pressure'],
 ['s/p'],
 ['disease'],
 ['artery', 'coronary'],
 ['pneumonia', 'sepsis'],
 ['pcp'],
 ['diabetes'],
 ['secondary'],
 ['chills', 'fever', 'fevers', 'shortness'],
 ['abdominal'],
 ['breath'],
 ['respiratory'],
 ['urinary'],
 ['incision', 'incisions'],
 ['left', 'right'],
 ['swimming'],
 ['breathing'],
 ['bleeding'],
 ['questions'],
 ['dry'],
 ['cancer'],
 ['with'],
 ['coumadin'],
 ['symptoms'],
 ['low'],
 ['aspirin'],
 ['antibiotics'],
 ['pulmonary'],
 ['wound'],
 ['tract'],
 ['kidney', 'renal'],
 ['drainage'],
 ['blood'],
 ['chest'],
 ['infection'],
 ['atrial', 'fibrillation'],
 ['acute', 'chronic'],
 ['lotions', 'ointments'],
 ['diet'],
 ['congestive', 'failure'],
 ['redness'],
 ['consciousness'],
 ['surgeon', 'surgery'],
 ['hours'],
 ['baths'],
 ['interactive']]

In [ ]:
len(condensed_feature)

51

In [ ]:
condensed_feature_dict={}
count=0
for i in condensed_feature:
  condensed_feature_dict['f'+str(count)]=i
  count+=1

In [ ]:
print(condensed_feature_dict)

{'f0': ['alert'], 'f1': ['medical'], 'f2': ['department', 'emergency'], 'f3': ['cardiac', 'heart'], 'f4': ['pain'], 'f5': ['pressure'], 'f6': ['s/p'], 'f7': ['disease'], 'f8': ['artery', 'coronary'], 'f9': ['pneumonia', 'sepsis'], 'f10': ['pcp'], 'f11': ['diabetes'], 'f12': ['secondary'], 'f13': ['chills', 'fever', 'fevers', 'shortness'], 'f14': ['abdominal'], 'f15': ['breath'], 'f16': ['respiratory'], 'f17': ['urinary'], 'f18': ['incision', 'incisions'], 'f19': ['left', 'right'], 'f20': ['swimming'], 'f21': ['breathing'], 'f22': ['bleeding'], 'f23': ['questions'], 'f24': ['dry'], 'f25': ['cancer'], 'f26': ['with'], 'f27': ['coumadin'], 'f28': ['symptoms'], 'f29': ['low'], 'f30': ['aspirin'], 'f31': ['antibiotics'], 'f32': ['pulmonary'], 'f33': ['wound'], 'f34': ['tract'], 'f35': ['kidney', 'renal'], 'f36': ['drainage'], 'f37': ['blood'], 'f38': ['chest'], 'f39': ['infection'], 'f40': ['atrial', 'fibrillation'], 'f41': ['acute', 'chronic'], 'f42': ['lotions', 'ointments'], 'f43': ['die

In [ ]:
all_vocab=[a for b in condensed_feature for a in b]

In [ ]:
diagnosis_icd9category['med_terms_condensed']=diagnosis_icd9category.med_terms_token.progress_apply(lambda x:list(set(x).intersection(set(all_vocab))))

In [ ]:
diagnosis_icd9category

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_token,med_terms_condensed
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","['discharge diagnosis', 'coronary artery disease/atypical angina', 'hypothyroidism', 'rca', 'cerebral vascular accident', 'questions', 'hypertension', 'discharge condition', 'productive cough', 'concerns', 'chest pain', 'hypercholesterolemia', 'discharge', 'hiatal hernia', 'rib fracture', 's/p motor vehicle', 'office', 'telephone']","[fracture, pain, rib, s/p, discharge, artery, accident, diagnosis, rca, hernia, productive, concerns, cerebral, hiatal, vehicle, telephone, questions, hypothyroidism, hypertension, angina, condition, disease/atypical, office, chest, coronary, hypercholesterolemia, vascular, motor, cough]","[questions, chest, pain, s/p, artery, coronary]"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**location (un) 1822**], [**apartment address(1) 1823**], [**location (un) **],[**numeric identifier 1700**]. the phone is <telephone>. you have an appointment to follow up with dr. <name> <name> in the <name> at <hospital> on the [**location (un) 453**]. their phone number is <telephone>. department: div. of gastroenterology when: wednesday <date> at 3:00 pm with: <name> <name>, md <telephone> building: ra <name> (<name>/<name> complex) [**location (un) **] campus: east best parki

In [ ]:
def return_feature(x):
  feature=[]
  for i in x:
    for j in list(condensed_feature_dict):
      if i in condensed_feature_dict[j]:
        feature.append(j)
  return list(set(feature))


In [ ]:
diagnosis_icd9category['features']=diagnosis_icd9category.med_terms_condensed.progress_apply(lambda x:return_feature(x))

In [ ]:
diagnosis_icdcategory_medterms_df['feature']=diagnosis_icdcategory_medterms_df.frequent_medterms.progress_apply(lambda x:return_feature(x))

In [ ]:
def predict_icd(x):
  category_weights={}
  if len(x):
    for item1 in frequent_icd9category:
      category_features=[a for b in diagnosis_icdcategory_medterms_df.loc[diagnosis_icdcategory_medterms_df.icd9category==item1].feature.to_list() for a in b]
      count=len(list(set(x).intersection(set(category_features))))
      category_weights[item1]=count/len(x)/len(category_features)
  else:
    for item1 in frequent_icd9category:
      category_weights[item1]=0.0

      
  return category_weights
  

In [ ]:
diagnosis_icd9category['predicted_icd_weigths']=diagnosis_icd9category.features.progress_apply(lambda x:predict_icd(x))

In [ ]:
diagnosis_icd9category

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,med_terms_token,med_terms_condensed,features,predicted_icd_weigths
0,167118.0,"discharge diagnosis: copd, coronary artery disease/atypical angina (lad 30%, rca 30%, ef 63%), hypercholesterolemia, hypothyroidism, hypertension, hiatal hernia, cerebral vascular accident,s/p motor vehicle colision-> head injury & rib fracture. tbm- s/p tracheoplasty. discharge condition: good discharge instructions: please update dr.<name> <telephone> office for: fever, shortness of breath, chest pain , productive cough or if you have any questions or concerns. completed by: <date>","[496, 553, 518, 453, 519, 276]","['discharge diagnosis', 'coronary artery disease/atypical angina', 'hypothyroidism', 'rca', 'cerebral vascular accident', 'questions', 'hypertension', 'discharge condition', 'productive cough', 'concerns', 'chest pain', 'hypercholesterolemia', 'discharge', 'hiatal hernia', 'rib fracture', 's/p motor vehicle', 'office', 'telephone']","[fracture, pain, rib, s/p, discharge, artery, accident, diagnosis, rca, hernia, productive, concerns, cerebral, hiatal, vehicle, telephone, questions, hypothyroidism, hypertension, angina, condition, disease/atypical, office, chest, coronary, hypercholesterolemia, vascular, motor, cough]","[questions, chest, pain, s/p, artery, coronary]","[f4, f23, f8, f38, f6]","{'401': 0.04, '427': 0.04, '276': 0.015384615384615385, '272': 0.038461538461538464, '250': 0.03076923076923077, '414': 0.04, '285': 0.022222222222222223, '428': 0.03333333333333333, '518': 0.015384615384615385, '584': 0.016, 'V45': 0.032, 'V58': 0.022222222222222223, '585': 0.02608695652173913, '530': 0.032, '599': 0.016, 'E87': 0.04, '403': 0.02608695652173913, 'V10': 0.03076923076923077, '038': 0.016, '995': 0.016}"
1,196489.0,"discharge diagnosis: primary: 1. chronic obstructive pulmonary disease exacerbation 2. respiratory failure with intubation 3. upper gastrointestinal bleed/peptic ulcer disease 4. hypertension 5. anxiety secondary: 1. coronary artery disease 2. hypothyroidism discharge condition: mental status: confused - sometimes. level of consciousness: alert and interactive. activity status: out of bed with assistance to chair or wheelchair. discharge instructions: you were admitted to the hospital with shortness of breath and respiratory failure and lethargy. in the emergency department, you were breathing very fast and a breathing tube was inserted into your airway to help you breathe. you were admitted to the intensive care unit. there, you were managed with steroids and antibiotics and the tube was eventually removed from your airway, allowing you to breathe on your own. you also underwent bronchoscopy which looked at the inside of your airways. . during your time in the intensive care unit, you developed a gastrointestinal bleed. a tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach. this showed inflammation in your esophagusand ulcers in your stomach. . you should continue to use nasal oxygen by nasal cannula as needed. you should continue the steroid taper as instructed. you should call your doctor or return to the emergency room if you have increasing difficulty breathing or shortness of breath, wheezing, chest pain, blood in your stool or vomiting blood. . there are some changes in your medications. start pantoprazole 40 mg twice daily and stop omeprazole start hydrochlorothiazide start prednisone, taking 30 mg for 3 days, then 20 mg for 4 days, then 10 mg for 4 days, then 5 mg for 4 days, then stop. start colace and senna as needed for constipation can use morphine to alleviate symptoms of respiratory discomfort stop furosemide stop zolpidem stop aspirin increase diovan to 240 mg daily decrease trazodone to 50 mg daily . follow up as indicated below. followup instructions: you have an appointment to follow up with dr. <name>, your primary care physician, <name> [**8-29**] at 1pm. his address is [**loca

In [ ]:
diagnosis_icd9category['predicted_icd']=diagnosis_icd9category.predicted_icd_weigths.progress_apply(predictbasedonwts)

In [ ]:
result_predict={'category':[],'precision':[],'recall':[],'f1':[]}
for i in frequent_icd9category:
  actual_p=diagnosis_icd9category.loc[diagnosis_icd9category.icd9_category.apply(lambda x:i in x)].HADM_ID.count()
  tp=diagnosis_icd9category.loc[(diagnosis_icd9category.icd9_category.apply(lambda x:i in x))&(diagnosis_icd9category.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
  predict_p=diagnosis_icd9category.loc[diagnosis_icd9category.predicted_icd.apply(lambda x:i in x)].HADM_ID.count()
  precision=tp/predict_p
  recall=tp/actual_p
  f1=2*precision*recall/(precision+recall)
  result_predict['category'].append(i)
  result_predict['precision'].append(precision)
  result_predict['recall'].append(recall)
  result_predict['f1'].append(f1)

In [ ]:
pd.DataFrame.from_dict(result_predict)

,category,precision,recall,f1
0,401,0.482773,0.578809,0.526447
1,427,0.367896,0.620706,0.461976
2,276,0.403920,0.530846,0.458766
3,272,0.393874,0.426916,0.409730
4,250,0.338874,0.546093,0.418223
5,414,0.367941,0.543950,0.438959
6,285,0.319031,0.482206,0.384003
7,428,0.333801,0.693245,0.450624
8,518,0.345443,0.530521,0.418430
9,584,0.308929,0.705093,0.429623


In [ ]:
total_predicted=len([a for b in diagnosis_icd9category.predicted_icd.to_list() for a in b])
print(total_predicted)
actual=len([a for b in diagnosis_icd9category.icd9_category.apply(lambda x : [i for i in x if i in frequent_icd9category]).to_list() for a in b])
print(actual)
actual_predicted=0
for i in frequent_icd9category:
  actual_predicted+=diagnosis_icd9category.loc[(diagnosis_icd9category.icd9_category.apply(lambda x:i in x))&(diagnosis_icd9category.predicted_icd.apply(lambda x:i in x))].HADM_ID.count()
print(actual_predicted)
recall=actual_predicted/actual
presicion=actual_predicted/total_predicted
print(recall)
print(presicion)
print(2*presicion*recall/(presicion+recall))

420331
189430
111842
0.5904133452990551
0.2660807791954436
0.3668388106159627
